In [1]:
"""
Feature: immunosuppressives_per_admission
Boolean flag indicating if immunosuppressives were given during admission
"""
import pandas as pd
import os
import re
from config_paths import *
from utils import drop_columns

In [2]:
# Read dataset
df = pd.read_csv(dataset_path)
initial_row_count = len(df)

# Load prescriptions
prescriptions = pd.read_csv(
    os.path.join(hosp_path, "prescriptions.csv"),
    usecols=["subject_id", "hadm_id", "drug"]
)

# Immunosuppressive pattern
immunosuppressive_pattern = r"(?:tacro|cyclospor|sirolim|everolim|azathi|mycophen|methotrex|ritux|inflix|adalim|etanercept|belatacept|globulin)"

def drug_flag_regex(df, pattern):
    return df["drug"].str.contains(pattern, case=False, na=False, regex=True)

# Build immunosuppressives flag per admission
immunosuppressives_flag_df = (
    prescriptions.assign(
        immunosuppressives=lambda x: drug_flag_regex(x, immunosuppressive_pattern)
    )
    .groupby(["subject_id", "hadm_id"])["immunosuppressives"]
    .max()
    .reset_index()
)

# Merge
df = df.merge(immunosuppressives_flag_df, on=["subject_id", "hadm_id"], how="left")

# Fill missing with False
df["immunosuppressives_per_admission"] = (
    df["immunosuppressives"]
    .astype("boolean")
    .fillna(False)
    .astype(bool)
)

# Drop the temporary immunosuppressives column
df = df.drop(columns=["immunosuppressives"])

# Save
df.to_csv(dataset_path, index=False)

# Diagnostic output
print(f"\nTotal rows: {len(df)}")
print(f"Expected total: {initial_row_count}")
if len(df) == initial_row_count:
    print(f"✓ Row count matches expected total ({initial_row_count})")
else:
    print(f"⚠ Row count mismatch! Expected {initial_row_count}, got {len(df)} (difference: {len(df) - initial_row_count})")
print(f"Dataset shape: {df.shape}")
print(f"Feature 'immunosuppressives_per_admission' added. True count: {df['immunosuppressives_per_admission'].sum()}")
print(f"Dataset shape: {df.shape}")




Total rows: 158020
Expected total: 158020
✓ Row count matches expected total (158020)
Dataset shape: (158020, 75)
Feature 'immunosuppressives_per_admission' added. True count: 7562
Dataset shape: (158020, 75)


In [3]:
df.columns

Index(['subject_id', 'hadm_id', 'cauti_type', 'cauti_icd_codes',
       'other_uti_icd_codes', 'remaining_icd_codes', 'gender', 'anchor_age',
       'admittime', 'dischtime', 'admission_type', 'admission_location',
       'discharge_location', 'race', 'catheter_procedures',
       'catheter_procedures_ir', 'catheter_used_in_procedures_ir',
       'catheter_insertion_date_ir', 'catheter_removal_date_ir',
       'catheter_outputevents', 'catheter_used_in_output_events',
       'datetimeevents', 'catheter_used_in_datetime_events',
       'insertion_date_from_datetimeevents',
       'removal_date_from_datetimeevents', 'catheter_procedure_events',
       'catheter_used_in_procedure_events', 'catheter_size_from_chartevents',
       'index', 'catheter_clinical_notes', 'catheter_used_in_clinical_notes',
       'catheter_present', 'final_cauti_flag', 'final_insertion_date',
       'final_removal_date', 'catheter_duration_days', 'BMI_in_admission',
       'BMI_computed', 'BMI_last_year', 'BMI', 

In [4]:
# cols_to_drop = [
#  "immunosuppressives_per_admission"
# ]
# df = drop_columns(df, cols_to_drop)
# df.to_csv(dataset_path, index=False)